In [1]:
import Datapull_Functions as DP

# API Setup
BASE_URL = 'https://api.collegefootballdata.com/'
with open("API_KEY.txt","r") as file:
    API_KEY = file.read()
DB_PATH = "cfb_data.db"

In [2]:
# Games Data Pull
games_df = DP.pull_initial_games_data(BASE_URL, API_KEY)

#Setup SQLite DB and save games data
DP.setup_sqlite_database(games_df, DB_PATH)

# Pull Lines Data
lines_df = DP.pull_lines_data(BASE_URL, API_KEY)
DP.save_lines_to_database(lines_df, DB_PATH)

# Average Betting Data
DP.average_betting_data(DB_PATH)

# Pull Advanced Stats
advanced_stats_df = DP.pull_advanced_stats_data(BASE_URL, API_KEY)
DP.save_advanced_stats_to_db(advanced_stats_df, DB_PATH)

# Merge Games and Advanced Stats
DP.merge_games_and_advanced_stats(DB_PATH)

# Pull Games/Teams Data
teams_stats_df = DP.pull_games_teams_data(BASE_URL, API_KEY)
DP.save_games_teams_data_to_db(teams_stats_df, DB_PATH)

# Merge with Games_with_stats
DP.merge_games_teams_data(DB_PATH)

# Pull returning production data
returning_df = DP.pull_returning_production_data(BASE_URL, API_KEY)
DP.save_returning_production_data_to_db(returning_df, DB_PATH)

Pulling games for 2013...
Pulling games for 2014...
Pulling games for 2015...
Pulling games for 2016...
Pulling games for 2017...
Pulling games for 2018...
Pulling games for 2019...
Pulling games for 2020...
Pulling games for 2021...
Pulling games for 2022...
Pulling games for 2023...
Pulling games for 2024...
Pulling games for 2025...
Pulling lines for 2013...
Pulling lines for 2014...
Pulling lines for 2015...
Pulling lines for 2016...
Pulling lines for 2017...
Pulling lines for 2018...
Pulling lines for 2019...
Pulling lines for 2020...
Pulling lines for 2021...
Pulling lines for 2022...
Pulling lines for 2023...
Pulling lines for 2024...
Pulling lines for 2025...
(35086, 8)
(11165, 35)
Pulling advanced stats for 2013...
Pulling advanced stats for 2014...
Pulling advanced stats for 2015...
Pulling advanced stats for 2016...
Pulling advanced stats for 2017...
Pulling advanced stats for 2018...
Pulling advanced stats for 2019...
Pulling advanced stats for 2020...
Pulling advanced stat